In [1]:
import pandahouse as ph

# Задание 2.1

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [3]:
query_2_1 = '''
WITH t1 AS (SELECT st_id, DATE_TRUNC('month', timest) as date, COUNT(*) count_pea
FROM default.peas
where correct = 1
GROUP BY st_id, date
HAVING COUNT(*) >= 20)
SELECT count(*) AS Number_of_diligent_students
FROM t1
    '''

In [4]:
Number_of_diligent_students = ph.read_clickhouse(query=query_2_1, connection=connection_default)
Number_of_diligent_students

,Number_of_diligent_students
0,136


# Задание 2.2

In [5]:
query_2_2 = '''

WITH activ_studs AS (SELECT DISTINCT st_id
                     FROM default.peas
                     WHERE correct = 1
                     GROUP BY st_id
                     HAVING COUNT(*) > 10),
count_activ_by_grp as (SELECT count(st_id) as count_st_id, test_grp 
                        FROM
                        (SELECT DISTINCT st_id
                        FROM default.peas
                        WHERE correct = 1
                        GROUP BY st_id
                        HAVING COUNT(*) > 10) as t1 
                        LEFT JOIN
                        default.studs ON t1.st_id = default.studs.st_id
                        group by test_grp),
activ_studs_math AS (SELECT DISTINCT st_id
                     FROM default.peas
                     WHERE correct = 1 and subject = 'Math'
                     GROUP BY st_id
                     HAVING COUNT(*) >= 2),

ARPU AS (SELECT test_grp,
                         SUM(money) / COUNT(DISTINCT st_id) AS ARPU
        FROM default.studs LEFT JOIN default.final_project_check ON default.final_project_check.st_id = default.studs.st_id
                          GROUP BY test_grp),
ARPAU AS (SELECT t1.test_grp, t1.sum_money / count_activ_by_grp.count_st_id as ARPAU
            FROM (SELECT test_grp, SUM(money) as sum_money
            FROM default.studs INNER JOIN default.final_project_check 
                   ON default.final_project_check.st_id = default.studs.st_id
            where st_id IN (select st_id FROM activ_studs)
            GROUP BY test_grp) as t1
            JOIN count_activ_by_grp USING(test_grp)),
CR AS (SELECT test_grp,countIf(DISTINCT final_project_check.st_id, final_project_check.st_id != '') / 
        COUNT(DISTINCT st_id) AS CR
FROM default.studs LEFT JOIN default.final_project_check 
                   ON default.final_project_check.st_id = default.studs.st_id
GROUP BY test_grp),
CR_FOR_ACTIV AS (SELECT test_grp, 
                        countIf(DISTINCT final_project_check.st_id, final_project_check.st_id != '') / COUNT(DISTINCT default.studs.st_id) as CR_FOR_ACTIV
                 FROM default.studs LEFT JOIN default.final_project_check ON default.studs.st_id = default.final_project_check.st_id
                 WHERE default.studs.st_id IN (SELECT st_id FROM activ_studs)
                 GROUP BY test_grp),
CR_FOR_ACTIV_MAT AS (SELECT test_grp, 
                            countIf(DISTINCT final_project_check.st_id, final_project_check.st_id != '' and subject = 'Math')/COUNT(DISTINCT default.studs.st_id) as CR_FOR_ACTIV_MAT
                     FROM default.studs LEFT JOIN default.final_project_check ON default.studs.st_id = default.final_project_check.st_id
                     WHERE default.studs.st_id IN (SELECT st_id FROM activ_studs_math)
                     GROUP BY test_grp)

SELECT ARPU.test_grp as test_grp,
       ARPU.ARPU as ARPU,
       ARPAU.ARPAU as ARPAU,
       CR.CR * 100 AS CR,
       CR_FOR_ACTIV.CR_FOR_ACTIV * 100 as CR_FOR_ACTIV,
       CR_FOR_ACTIV_MAT.CR_FOR_ACTIV_MAT * 100 AS CR_FOR_ACTIV_MAT
FROM CR JOIN CR_FOR_ACTIV  USING(test_grp)
                    JOIN CR_FOR_ACTIV_MAT ON CR.test_grp = CR_FOR_ACTIV_MAT.test_grp
                    JOIN ARPU ON CR_FOR_ACTIV_MAT.test_grp = ARPU.test_grp
                    JOIN ARPAU ON CR_FOR_ACTIV_MAT.test_grp = ARPAU.test_grp'''

In [6]:
Metrics = ph.read_clickhouse(query=query_2_2, connection=connection_default)
Metrics

,test_grp,ARPU,ARPAU,CR,CR_FOR_ACTIV,CR_FOR_ACTIV_MAT
0,control,4540.983607,10393.700787,4.918033,11.023622,6.122449
1,pilot,11508.474576,29739.583333,10.847458,26.041667,9.523810
